In [ ]:
from sklearn.model_selection import train_test_split
import os
import shutil

# Paths
original_dir = "/home/mahdi/logo_detection_model/test_augmentation/detected_logos"
train_dir = "/home/mahdi/logo_detection_model/test_augmentation/train"
val_dir = "/home/mahdi/logo_detection_model/test_augmentation/validation"
test_dir = "/home/mahdi/logo_detection_model/test_augmentation/test"

# Ensure train, validation, and test folders exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Split data by class
for class_name in os.listdir(original_dir):
    class_path = os.path.join(original_dir, class_name)
    images = os.listdir(class_path)

    # Skip empty or invalid class directories
    if not images:
        print(f"Skipping empty class folder: {class_name}")
        continue

    # Create class-specific subdirectories
    train_class_path = os.path.join(train_dir, class_name)
    val_class_path = os.path.join(val_dir, class_name)
    test_class_path = os.path.join(test_dir, class_name)

    os.makedirs(train_class_path, exist_ok=True)
    os.makedirs(val_class_path, exist_ok=True)
    os.makedirs(test_class_path, exist_ok=True)

    # Split images
    train_images, temp_images = train_test_split(images, test_size=0.3, random_state=42)
    val_images, test_images = train_test_split(temp_images, test_size=0.5, random_state=42)

    # Copy files to their respective folders
    for img in train_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(train_class_path, img))
    for img in val_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(val_class_path, img))
    for img in test_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(test_class_path, img))

print("Dataset split completed.")


In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import tensorflow as tf
# Paths
original_dir = "/home/mahdi/logo_detection_model/test_augmentation/train"  # Path to the original images
augmented_dir = "/home/mahdi/logo_detection_model/test_augmentation/augmented_train_images"  # Temporary path for augmented images

os.makedirs(augmented_dir, exist_ok=True)

# Define augmentations
train_aug_rotate = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=25
    )

train_orginal = ImageDataGenerator(
    rescale = 1.0 / 255
)

train_aug_flip = ImageDataGenerator(
    rescale=1.0 / 255,
    horizontal_flip=True
    )

train_aug_gray = ImageDataGenerator(
    rescale=1.0 / 255,
      preprocessing_function=lambda img: tf.image.rgb_to_grayscale(img).numpy()
      )

# Function to save augmented images
def augment_and_save(datagen, source_dir, target_dir, prefix, num_gen):
    for class_name in os.listdir(source_dir):  # Loop through classes
        class_source_path = os.path.join(source_dir, class_name)
        class_target_path = os.path.join(target_dir, class_name)
        os.makedirs(class_target_path, exist_ok=True)

        for img_name in os.listdir(class_source_path):  # Loop through images
            img_path = os.path.join(class_source_path, img_name)
            img = load_img(img_path)
            x = img_to_array(img).reshape((1,) + img_to_array(img).shape)

            # Generate augmented images and save
            for batch, _ in zip(datagen.flow(x, batch_size=1, save_to_dir=class_target_path,
                                             save_prefix=prefix, save_format="jpg"), num_gen ):
                pass  # Generate 5 augmented images per original image

# Apply augmentations
augment_and_save(train_orginal, original_dir, augmented_dir, prefix="orginal", num_gen=range(0))
augment_and_save(train_aug_rotate, original_dir, augmented_dir, prefix="rotate", num_gen=range(0))
# augment_and_save(train_aug_flip, original_dir, augmented_dir, prefix="flip", num_gen=range(0))
augment_and_save(train_aug_gray, original_dir, augmented_dir, prefix="gray", num_gen=range(0))


In [ ]:
# train and test and validation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
# Paths
augmented_dir = "/home/mahdi/logo_detection_model/test_augmentation/augmented_train_images"
val_dir = "/home/mahdi/logo_detection_model/test_augmentation/validation"
test_dir = "/home/mahdi/logo_detection_model/test_augmentation/test"

# Combine original and augmented data
combined_datagen = ImageDataGenerator(
    rescale=1.0 / 255                             
    )

train_generator = combined_datagen.flow_from_directory(
    augmented_dir,             # Directory with both original and augmented images
    target_size=(224, 224),    # Resize to match model input
    batch_size=32,
    class_mode="binary",
    # subset="training"
)

# # Validation generator (no augmentation, just preprocessing)
# val_datagen = ImageDataGenerator(
#     rescale=1.0 / 255        
          
# )

# # Validation generator
# val_generator = val_datagen.flow_from_directory(
#     val_dir,
#     target_size=(224, 224),
#     batch_size=16,
#     class_mode="binary",
#     # subset="validation"
# )

# print(f"Training samples: {train_generator.samples}")
# print(f"Validation samples: {val_generator.samples}")







# No augmentation for validation/test
val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# # Create generators
# train_generator = train_datagen.flow_from_directory(
#     train_dir,
#     target_size=(224, 224),
#     batch_size=32,
#     class_mode="binary"
# )

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode="binary"
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode="binary"
)

print(f"Train samples: {train_generator.samples}")
print(f"Validation samples: {val_generator.samples}")
print(f"Test samples: {test_generator.samples}")

